# Jupyter Notebook Investment Planning Optimization models using Linear Programming in R

Description- Numerous opportunities are available to today's investor. In many of these situations Linear Programing can be used to select the optimal mix of opportunities that will maximize returns,In this second edition we walk through those scenarios 

Purpose: This notebook provides a template for solving optimization problems. It highlights how one can go about formulating the problem in order to use the functions within the lpsolve package to obtain a result that meets all of the given constraints

In the first edition we walk through a series of problems that mainly dealt with sales and operations. The question then become can this same methodology be applied in other industries? The following example are the author's attempts at proving the breadth of use for Optimization Models. Let's first start with the below example


  # Dependencies
  
  Load the package for model development
  * lpSolve




### BANK LOAN MODEL

Bank One is in the process of devising a loan policy that involves a maximum of $12 million. The following table provides the pertinent data about available loans.


In [19]:
#install.packages("lpsolve")
install.packages("lpSolve",repos="http://cran.rstudio.com/")
library(lpSolve)

package 'lpSolve' successfully unpacked and MD5 sums checked


Warning message:
"cannot remove prior installation of package 'lpSolve'"


The downloaded binary packages are in
	C:\Users\alarrieux001\AppData\Local\Temp\RtmpeGCb6T\downloaded_packages


ERROR: Error in library(lpSolve): there is no package called 'lpSolve'


In [ ]:
Loan_Table <- data.frame("Type of Loan" = c("Personal","Car","Home","Farm","Commercial"),
                         "Interest Rate" =c(".140",".130",".120",".125",".100"),
                          "Bad Debt Ratio"=c(".10",".07",".03",".05",".02"))

Loan_Table

#### Bad debts are unrecoverable and produce no interest revenue. Competition with other financial institutions dictates the allocation of at least 40% of the funds to farm and commercial loans. To assit the housing industry in the region, home loans must equal at least 50% of the personal, car, and home loans. As a business rule the bank limits the overall ratio of bad debts on all loans to at most 4%

In order to build our mathematical model, the situation deals with determining the amount of loan in each category, thus leading to the following definitions of the variables:

$x_{1}$ = Personal loans (in millions of dollars)

$x_{2}$ = car loans

$x_{3}$ = home loans

$x_{4}$ = Farm loans

$x_{5}$ = Commercial loans

### The objective of the bank is to maximize net return, the difference between interest revenue, and lost bad debts. Interest revenue is accrued on loands in good standing. For example, when 10% of personal loans are lost to bad debt, the bank will receive interest on 90% of the loan - that is, it will receive 14% interest on .9$x_{1}$ of the original $x_{1}$. The same reasoning applies to the remaining four types of loans. Leading to the below


Total Interest = .14(.9$x_{1}$) +  .13(.93$x_{2}$) + .12(.97$x_{3}$) + .125(.95$x_{4}$) + .1(.98$x_{5}$) 

Total Interest = .126$x_{1}$ + .1209$x_{2}$ + .1164$x_{3}$ + .11875$x_{4}$ + .098$x_{5}$

We also have 

Bad Debt = .1$x_{1}$ + .07$x_{2}$ + .03$x_{3}$ + .05$x_{4}$ + .02$x_{5}$

The objective function combines revenue and bad debt as:

Maximize Z = Total Interest - Bad Debt

Max(z) = .026$x_{1}$ + .0509$x_{2}$ + .1164$x_{3}$ + .05$x_{4}$ + .02$x_{5}$

###### The problem has 5 constraints


###### 1. Total funds should not exceed 12 million:

$x_{1}$ +   $x_{2}$ +   $x_{3}$ +   $x_{4}$ +   $x_{5}$  <= 12


###### 2. Farms and commercial loans equal at least 40% of all loans:
Which can be written as either 

$x_{4}$ + $x_{5}$ >= .4($x_{1}$+$x_{2}$+$x_{3}$+$x_{4}$+$x_{5}$)

or

4($x_{1}$)+4($x_{2}$)+4($x_{3})$-6($x_{4}$)-6($x_{4}$) <= 0


###### 3. Home loans should equal at least 50% of personal, car, and home loans:

$x_{3}$ >= .5($x_{1}$ + $x_{2}$ + $x_{3}$)

or

.5$x_{1}$ + .5$x_{2}$ + .5$x_{3}$ <= 0

###### 4. Bad debts should not exceed 4 % of all loans:
.1$x_{1}$ + .7$x_{2}$ + .03$x_{3}$ + .05$x_{4}$ + .02$x_{5}$ >= .4($x_{1}$ + $x_{2}$ + $x_{3}$ + $x_{4}$ + $x_{5}$)

or

.06$x_{1}$ + .03$x_{2}$ - .01$x_{3}$ + .01$x_{4}$ - $x_{5}$ <= 0


###### 5. Nonnegativity:
$x_{1}$, $x_{2}$ ,$x_{3}$ ,$x_{4}$ ,$x_{5}$  >= 0


A subtle assumption in the preceding formulation is that all loans are issued at approximately the same time. This allows us to ignore differences in the time value of the funds allocated to the different loans.



In [25]:
#we should have the lpsolve package install and loaded if now please include the below
#install.packages("lpSolve",repos="http://cran.rstudio.com/")
#library(lpSolve)
#Maximize Z = Total Interest - Bad Debt
objective.in <- c(.026,0.0509,.0864,.06875,.078)



In [26]:
const.mat <- matrix(c(1,1,1,1,1,.4,.4,.4,-.6,-.6,.5,.5,-.5,0,0,.06,.03,-.01,.01,-.02,1,1,1,1,1),ncol=5, byrow=TRUE)

In [27]:
condir <- c("<=","<=","<=","<=",">=")

In [28]:
conrhs <- c(12000000,0,0,0,0)

In [29]:
prod.sol <- lpSolve::lp(direction='max',objective.in,const.mat,condir,conrhs,compute.sen= TRUE)

In [30]:
prod.sol

Success: the objective function is 996480 

In [31]:
prod.sol$solution

[1]       0       0 7200000       0 4800000

# Remarks

While the answer is the recommended allocation of the funds, note that removing some contraints would lead to a different answer. For example the optimal solution calls for the allocation of the 12 million: 7.2 million to home loans and 4.8 million to commercial loans. The remaining categories receive non. The return on investment then becomes

Rate of return = Z/12 = .99648/12 = .0834

The above of .0834 is less than the best net interest rate = 8.64% for home loans. The question one may wonder is why the model did not take full advantage of this opportunity. The answer is that the stipulation that farm and commercial must account for at least 40% of all loans( constraint 2) forces the solution to allocate 4.8 million to commercial loans at the lower net rate of 7.8%, lowering the overall interest rate

### The second problem we will take a look at is a manufacturer looking to take advantage of fluctuation in manufacturing cost.

### Multiple Period Production- Inventory Model


ACME Manufacturing Company has a contract to delivery 100, 250, 190, 140, 220, 110 home windows over the next 6 months. Production Cost ( Labor, Material, and Utilities) per window varies by period and is estimated to be $50, $45, $55, $48, $52, and $50 over the next 6 months. To take advantage of the fluctuations in manufacturing cost, Acme can produce more windows than needed in a given month and hold the extra units for delivery in later months. This will incur storage cost at the rate of $8 per window per month, assessed on end-of-month inventory. We will developa linear program to determine the optimum production schedule.


The variables of the problem include the monthly production amount and the end-of- month inventory. For ${i}$ = 1,2...6 let

$X_{i}$ = Number of Units produced in month ${i}$

$I_{i}$ = Inventory units left at the end of month ${i}$

Total Production Cost = 50$x_{1}$+45$x_{2}$+55$x_{3}$+48$x_{4}$+52$x_{5}$+50$x_{6}$

Total Inventory(Storage) Cost =8($I_{1}$+$I_{2}$+$I_{3}$+$I_{4}$+$I_{5}$+$I_{6}$)

###### The objective function then becomes:

###### Minimize Z = 50$x_{1}$+45$x_{2}$+55$x_{3}$+48$x_{4}$+52$x_{5}$+50$x_{6}$+8($I_{1}$+$I_{2}$+$I_{3}$+$I_{4}$+$I_{5}$+$I_{6}$)

The constraints of the problem can be represented as 

Beginning inventory + Production amount - Ending inventory = Demand

This is translated mathematically for the individuals months as 

$x_{1}$ - $I_{1}$ = 100 (Month 1)

$I_{1}$+ $x_{2}$ - $I_{2}$  = 250 (Month 2)

$I_{2}$+ $x_{3}$ - $I_{3}$  = 190 (Month 3)

$I_{3}$+ $x_{4}$ - $I_{4}$ = 140 (Month 4)

$I_{4}$+ $x_{5}$ - $I_{5}$ = 220 (Month 5)

$I_{5}$ + $x_{6}$ = 110 (Month 6)



In [66]:
objective.in = c(50,45,55,48,52,50,8,8,8,8,8,8)



In [67]:
const.mat = matrix(c(1,0,0,0,0,0,-1,0,0,0,0,0,
              0,1,0,0,0,0,1,-1,0,0,0,0,
              0,0,1,0,0,0,0,1,-1,0,0,0,
              0,0,0,1,0,0,0,0,1,-1,0,0,
              0,0,0,0,1,0,0,0,0,1,-1,0,
              0,0,0,0,0,1,0,0,0,0,1,0), ncol = 12, byrow = TRUE)

In [68]:
const.mat

1,0,0,0,0,0,-1,0,0,0,0,0
0,1,0,0,0,0,1,-1,0,0,0,0
0,0,1,0,0,0,0,1,-1,0,0,0
0,0,0,1,0,0,0,0,1,-1,0,0
0,0,0,0,1,0,0,0,0,1,-1,0
0,0,0,0,0,1,0,0,0,0,1,0


In [69]:
condir = c("=","=","=","=","=","=")

In [70]:
conrhs =c(100,250,190,140,220,110)

In [71]:
prod.sol <- lpSolve::lp(direction='min',objective.in,const.mat,condir,conrhs,compute.sen= TRUE)

In [72]:
prod.sol

Success: the objective function is 49980 

In [73]:
prod.sol$solution

[1] 100 440   0 140 220 110   0 190   0   0   0   0

In [89]:
Timeline <- data.frame( 
        Produced= c(100,440,0,140,220,110),
        Carryover = c(0,190,0,0,0,0)
    )


In [90]:
Timeline

x,i
100,0
440,190
0,0
140,0
220,0
110,0


# Remarks

The above recommends that, based on cost of production, it is more cost effective to overproduce in month two and carry over an additional 190 in order to meet the demand for month three. Note while this may be the recommended solution, your knowledge of the business should be used in order to make the end decision that provides the most benefit.

### Finally we look at an Oil refinery with the goal of determining the optimal mex of final products that will maximize profit.

### Crude oil refining and gasoline blending.

ABC Oil has a capacity of 1,500,000 bbl of crude oil per day. The final products from the refinery include three types of unleaded gasoline with different octane numbers (ON): 
        Regular = 87
        Premium = 89
        Super   = 92

The refining process encompases three stages: 
1. A distilation tower that provides feedstock (ON=82) at the rate of .2 bbl per bbl of crude oil
2. A cracker unit that produces gasoline (ON = 98) by using a portion of the feedstock produced from the distilation tower at the rate of .5 bbl per bbl of feedstock.
3. A blender unit that blends the gasoline stock from the cracker unit and the feedstock from the distilation tower.

The company estimates the net profit per barrel of the the three types to be:

1. 87 = $6.70

2. 89 = $7.20

3. 92 = $8.10

The input capacity is 200,000 bbl of feedstock per day. The demand limits for each are
1. 87 = 50,000

2. 89 = 30,000

3. 92 = 40,000

The objective function then becomes to maximize profits(z) = 6.70 $x_{11}$ + $x_{21}$)+7.20($x_{12}$ + $x_{22}$)+8.10($x_{13}$ + $x_{23}$)

The constraints can be represented as 

5($x_{11}$ + $x_{12}$+ $x_{13}$) + 10($x_{21}$ + $x_{22}$ + $x_{23}$) <= 1,500,000

2($x_{21}$ + $x_{22}$ + $x_{23}$) <= 200,000

$x_{11}$ + $x_{21}$ <= 50,000

$x_{12}$ + $x_{22}$ <= 30,000

$x_{13}$ + $x_{23}$ <= 40,000

5* $x_{11}$ - 11 * $x_{21}$ <= 0

7* $x_{12}$ - 9 * $x_{22}$ <= 0

10* $x_{12}$ - 6 * $x_{22}$ <= 0

In [1]:
objective.in = c(6.7, 7.2, 8.1,6.7,7.2,8.1)

In [2]:
const.mat = matrix(c(5,5,5,10,10,10,
                     0,0,0,2,2,2,
                     1,0,0,1,0,0,
                     0,1,0,0,1,0,
                     0,0,1,0,0,1,
                     5,0,0,-11,0,0,
                     0,7,0,0,-9,0,
                     0,0,10,0,0,-6),ncol = 6, byrow = TRUE)

In [3]:
condir = rep("<=",8)

In [4]:
conrhs = c(1500000,200000,50000,30000,40000,0,0,0)

In [5]:
prod.sol <- lpSolve::lp(direction='max',objective.in,const.mat,condir,conrhs,compute.sen= TRUE)

ERROR: Error in loadNamespace(name): there is no package called 'lpSolve'


In [9]:
prod.sol

Success: the objective function is 875000 

In [10]:
prod.sol$solution

[1] 34375 16875 15000 15625 13125 25000

In [17]:
solution_breakdown = data.frame(
                     DailyProfit       = 87500,
                    Daily.amount.Regular= 30000,
                    Daily.amount.Premium= 30000,
                    Daily.amount.Super   =40000)

In [18]:
solution_breakdown

DailyProfit,Daily.amount.Regular,Daily.amount.Premium,Daily.amount.Super
87500,30000,30000,40000


###Congrats!! you've gone through another list of optimization problems. This time we focused on investment planning, especially in the current environment, reading Operational Research by TAHA made sense to illustrate them in the above format. Feel free to check out my previous version at https://lab.pwc.com/automation/details/12651